# Лабораторная работа №5 (Проведение исследований с градиентным бустингом)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

In [2]:
import warnings

# Игнорировать все предупреждения
warnings.filterwarnings('ignore')

In [3]:
df_class = pd.read_csv('classification.csv')
df_reg = pd.read_csv('regression.csv')

Датасет для классификации (Air Quality & Health Impact Analysis):
* **RecordID:** Уникальный идентификатор, присваиваемый каждой записи
* **AQI:** Индекс качества воздуха, показывающий, насколько загрязнен воздух в настоящее время или насколько загрязненным он, по прогнозам, станет в будущем
* **PM10**:  Концентрация твердых частиц диаметром менее 10 микрометров (μg/m³)
* **PM2_5**: Концентрация твердых частиц диаметром менее 2,5 микрометров (μg/m³)
* **NO2**: Концентрация диоксида азота (ppb)
* **SO2**: Концентрация диоксида серы (ppb)
* **O3**: Концентрация озона (ppb)
* **Temperature**: Температура в градусах Цельсия (°C)
* **Humidity**: Процент влажности (%)
* **WindSpeed**: Скорость ветра (m/s)
* **RespiratoryCases**: Количество зарегистрированных респираторных случаев.
* **CardiovascularCases**: Количество зарегистрированных сердечно-сосудистых случаев
* **HospitalAdmissions**: Количество зарегистрированных случаев госпитализации
* **Target Variable: HealthImpactClass**

Датасет для регрессии (Electrity Prices):
* **DateTime**: дата и время
* **Holiday**: название праздника, если день нерабочий день
* **HolidayFlag**: целое число, 1, если день нерабочий день, ноль в противном случае
* **DayOfWeek**: целое число (0-6), 0 понедельник, день недели
* **WeekOfYear**: текущая неделя в течение года, начинающегося с этой даты
* **Day integer**: день
* **Month integer**: месяц
* **Year integer**: год
* **PeriodOfDay**: период суток
* **ForecastWindProduction**: прогнозируемая мощность ветра на этот период
* **SystemLoadEA**: национальный прогноз нагрузки на этот период
* **SMPEA**: прогноз цен на данный период
* **ORKTemperature**: фактическая температура
* **ORKWindspeed**: фактическая скорость ветра
* **CO2Intensity**: фактическая интенсивность выбросов CO2 в произведенной электроэнергии (г/кВт*ч)
* **ActualWindProduction**: фактическая нагрузка на национальную систему за этот период
* **SystemLoadEP2**: фактическая цена за данный период времени, прогнозируемое значение.
* **Target Variable: SystemLoadEP2**

## Создание бейзлайна

In [4]:
def simple_classification(df):
  X_class = df.drop(['HealthImpactClass','HealthImpactScore','RecordID'], axis=1)
  y_class = df['HealthImpactClass']

  X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(X_class, y_class, test_size=0.3, random_state=42)

  return X_train_class, X_test_class, y_train_class, y_test_class

In [5]:
X_train, X_test, y_train, y_test = simple_classification(df_class)

model = GradientBoostingClassifier()

model.fit(X_train, y_train)

predictions = model.predict(X_test)

# Оценка качества модели
accuracy_class = accuracy_score(y_test, predictions)
print(f"Точность GradientBoostingClassifier: {accuracy_class:.4f}")

Точность GradientBoostingClassifier: 0.8899


### Регрессия

In [6]:
def simple_regression(df):
  X_reg = df.drop(['SMPEP2', 'DateTime','Holiday'], axis=1, errors='ignore')
  y_reg = df['SMPEP2']

  for col in X_reg.columns:
      X_reg[col] = pd.to_numeric(X_reg[col], errors='coerce')

  y_reg = pd.to_numeric(y_reg, errors='coerce')

  X_reg = X_reg.fillna(0)
  y_reg = y_reg.fillna(0)

  X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_reg, y_reg, test_size=0.3, random_state=42)

  return X_train_reg, X_test_reg, y_train_reg, y_test_reg

In [7]:
X_train, X_test, y_train, y_test = simple_regression(df_reg)

model = GradientBoostingRegressor()
model.fit(X_train, y_train)

predictions = model.predict(X_test)

mse_reg = mean_squared_error(y_test, predictions)
print(f"Среднеквадратичная ошибка GradientBoostingRegressor: {mse_reg:.4f}")

Среднеквадратичная ошибка GradientBoostingRegressor: 582.0431


## Улучшение бейзлайна

In [8]:
def upgraded_classification(df):
  df = df.drop(columns=["RecordID"], errors="ignore")

  low_PM2_5 = df["PM2_5"].quantile(0.25)
  strong_PM2_5 = df["PM2_5"].quantile(0.75)
  df["LowPM2_5"] = (df["PM2_5"] <= low_PM2_5).astype(int)
  df["StrongPM2_5"] = (df["PM2_5"] >= strong_PM2_5).astype(int)

  low_PM10 = df["PM10"].quantile(0.25)
  strong_PM10 = df["PM10"].quantile(0.75)
  df["LowPM10"] = (df["PM10"] <= low_PM10).astype(int)
  df["StrongPM10"] = (df["PM10"] >= strong_PM10).astype(int)


  low_AQI = df["AQI"].quantile(0.25)
  strong_AQI = df["AQI"].quantile(0.75)
  df["LowAQI"] = (df["AQI"] <= low_AQI).astype(int)
  df["StrongAQI"] = (df["AQI"] >= strong_AQI).astype(int)

  df = df.drop(columns=["RespiratoryCases", "CardiovascularCases","WindSpeed","Temperature","Humidity",'PM2_5'], axis=1)

  df = df[[
      "HealthImpactClass",
      "LowAQI",
      "LowPM2_5",
      "LowPM10",
      "StrongPM2_5",
      "PM10",
      "O3",
      "StrongAQI",
      "AQI",
      "HealthImpactScore"
  ]]

  X_class = df.drop(['HealthImpactClass','HealthImpactScore'], axis=1)
  y_class = df['HealthImpactClass']

  X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(X_class, y_class, test_size=0.3, random_state=42)

  return X_train_class, X_test_class, y_train_class, y_test_class

In [16]:
X_train, X_test, y_train, y_test = upgraded_classification(df_class)

In [17]:
gb = GradientBoostingClassifier(random_state=42)

param_dist = {
    "n_estimators": randint(50, 400),
    "learning_rate": uniform(0.01, 0.3),
    "max_depth": randint(2, 10),
    "min_samples_split": randint(2, 30),
    "min_samples_leaf": randint(1, 20),
}


random_search = RandomizedSearchCV(
    estimator=gb,
    param_distributions=param_dist,
    n_iter=10,
    cv=5,
    scoring="accuracy",
    random_state=42,
    n_jobs=-1
)


random_search.fit(X_train, y_train)


print("Лучшие параметры:", random_search.best_params_)
print("Лучший score:", random_search.best_score_)

print("Точность на тесте:", random_search.score(X_test, y_test))

Лучшие параметры: {'learning_rate': np.float64(0.16742692948967136), 'max_depth': 5, 'min_samples_leaf': 17, 'min_samples_split': 28, 'n_estimators': 108}
Лучший score: 0.8785343209697454
Точность на тесте: 0.8715596330275229


### Регрессия

In [18]:
def upgraded_regression(df):
  df['ForecastWindProduction'] = pd.to_numeric(df['ForecastWindProduction'], errors='coerce')
  df['SystemLoadEA'] = pd.to_numeric(df['SystemLoadEA'], errors='coerce')
  df['SMPEA'] = pd.to_numeric(df['SMPEA'], errors='coerce')
  df['ORKTemperature'] = pd.to_numeric(df['ORKTemperature'], errors='coerce')
  df['ORKWindspeed'] = pd.to_numeric(df['ORKWindspeed'], errors='coerce')
  df['CO2Intensity'] = pd.to_numeric(df['CO2Intensity'], errors='coerce')
  df['ActualWindProduction'] = pd.to_numeric(df['ActualWindProduction'], errors='coerce')
  df['SystemLoadEP2'] = pd.to_numeric(df['SystemLoadEP2'], errors='coerce')
  df['SMPEP2'] = pd.to_numeric(df['SMPEP2'], errors='coerce')

  df = df.drop(["DateTime","Holiday"],axis = 1)

  df = df.dropna()

  df = df[df['SMPEP2'] > 0]
  df = df[df['SMPEP2'] != 1000]

  X_reg = df.drop(['SMPEP2'], axis=1, errors='ignore')
  y_reg = df['SMPEP2']

  X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_reg, y_reg, test_size=0.3, random_state=42)

  return X_train_reg, X_test_reg, y_train_reg, y_test_reg

In [19]:
X_train, X_test, y_train, y_test = upgraded_regression(df_reg)

In [20]:
gbr = GradientBoostingRegressor(random_state=42)

param_dist = {
    "n_estimators": randint(50, 400),
    "learning_rate": uniform(0.01, 0.3),
    "max_depth": randint(2, 10),
    "min_samples_split": randint(2, 30),
    "min_samples_leaf": randint(1, 20),
}

random_search = RandomizedSearchCV(
    estimator=gbr,
    param_distributions=param_dist,
    n_iter=5,
    cv=5,
    scoring="neg_mean_squared_error",
    random_state=42,
    n_jobs=-1
)

random_search.fit(X_train, y_train)

print("Лучшие параметры:", random_search.best_params_)
print("Лучший MSE:", -random_search.best_score_)

y_pred = random_search.predict(X_test)
print("MSE на тесте:", mean_squared_error(y_test, y_pred))

Лучшие параметры: {'learning_rate': np.float64(0.14777466758976016), 'max_depth': 6, 'min_samples_leaf': 4, 'min_samples_split': 9, 'n_estimators': 201}
Лучший MSE: 549.5792531861805
MSE на тесте: 504.2124933255265


## Имплементация алгоритма

In [21]:
class My_Decision_Tree_Regressor:
    def __init__(self,
                 max_depth=None,
                 min_samples_split=2,
                 min_samples_leaf=1):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.tree = None


    def _variance(self, y):
        return np.var(y) if len(y) > 0 else 0

    def _best_split(self, X, y):
        m, n = X.shape
        if m < self.min_samples_split:
            return None, None

        best_feat, best_thr = None, None
        best_var = 1e18

        total_sum = y.sum()
        total_sq_sum = np.dot(y, y)

        for feature in range(n):
            sorted_idx = np.argsort(X[:, feature])
            X_f = X[sorted_idx, feature]
            y_f = y[sorted_idx]

            left_sum = 0.0
            left_sq_sum = 0.0
            left_n = 0


            right_sum = total_sum
            right_sq_sum = total_sq_sum
            right_n = m

            for i in range(1, m):
                yi = y_f[i - 1]


                left_sum += yi
                left_sq_sum += yi * yi
                left_n += 1

                right_sum -= yi
                right_sq_sum -= yi * yi
                right_n -= 1


                if X_f[i] == X_f[i - 1]:
                    continue

                if left_n < self.min_samples_leaf or right_n < self.min_samples_leaf:
                    continue


                left_var = left_sq_sum / left_n - (left_sum / left_n)**2

                right_var = right_sq_sum / right_n - (right_sum / right_n)**2


                weighted_var = left_n * left_var + right_n * right_var

                if weighted_var < best_var:
                    best_var = weighted_var
                    best_feat = feature
                    best_thr = (X_f[i] + X_f[i - 1]) / 2

        return best_feat, best_thr

    def _build_tree(self, X, y, depth=0):
        m = len(y)

        if (self.max_depth is not None and depth >= self.max_depth) or \
           m < self.min_samples_split or \
           np.var(y) < 1e-10:
            return {"leaf": True, "value": np.mean(y)}

        feat, thr = self._best_split(X, y)

        if feat is None:
            return {"leaf": True, "value": np.mean(y)}

        left_idx = X[:, feat] <= thr
        right_idx = ~left_idx

        return {
            "leaf": False,
            "feature": feat,
            "threshold": thr,
            "left": self._build_tree(X[left_idx], y[left_idx], depth + 1),
            "right": self._build_tree(X[right_idx], y[right_idx], depth + 1)
        }


    def fit(self, X, y):
        X, y = np.array(X), np.array(y, dtype=float)
        self.tree = self._build_tree(X, y)
        return self

    def _predict_one(self, x, node):
        while not node["leaf"]:
            if x[node["feature"]] <= node["threshold"]:
                node = node["left"]
            else:
                node = node["right"]
        return node["value"]

    def predict(self, X):
        X = np.array(X)
        return np.array([self._predict_one(x, self.tree) for x in X])


class My_Gradient_Boosting_Classifier:
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.trees = []
        self.F0 = None

    def _sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def fit(self, X, y):
        y = y.astype(float)

        p = np.clip(y.mean(), 1e-6, 1 - 1e-6)
        self.F0 = np.log(p / (1 - p))

        F = np.full(y.shape, self.F0)

        for _ in range(self.n_estimators):
            prob = self._sigmoid(F)

            residual = y - prob

            tree = My_Decision_Tree_Regressor(max_depth=self.max_depth)
            tree.fit(X, residual)

            self.trees.append(tree)

            F += self.learning_rate * tree.predict(X)

    def predict_proba(self, X):
        F = np.full(X.shape[0], self.F0)

        for tree in self.trees:
            F += self.learning_rate * tree.predict(X)

        return self._sigmoid(F)

    def predict(self, X):
        return (self.predict_proba(X) >= 0.5).astype(int)

In [22]:
class My_Gradient_Boosting_Regressor:
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.trees = []
        self.F0 = None

    def fit(self, X, y):
        y = y.astype(float)

        self.F0 = y.mean()

        F = np.full(y.shape, self.F0)

        for _ in range(self.n_estimators):
            residual = y - F

            tree = My_Decision_Tree_Regressor(max_depth=self.max_depth)
            tree.fit(X, residual)

            self.trees.append(tree)

            F += self.learning_rate * tree.predict(X)

    def predict(self, X):
        F = np.full(X.shape[0], self.F0)

        for tree in self.trees:
            F += self.learning_rate * tree.predict(X)

        return F

In [23]:
X_train, X_test, y_train, y_test = simple_classification(df_class)

model = My_Gradient_Boosting_Classifier()

model.fit(X_train, y_train)

predictions = model.predict(X_test)

accuracy_class = accuracy_score(y_test, predictions)
print(f"Точность My_Gradient_Boosting_Classifier: {accuracy_class:.4f}")

Точность My_Gradient_Boosting_Classifier: 0.8578


In [24]:
X_train, X_test, y_train, y_test = simple_regression(df_reg)

model = My_Gradient_Boosting_Regressor()
model.fit(X_train, y_train)

predictions = model.predict(X_test)

mse_reg = mean_squared_error(y_test, predictions)
print(f"Среднеквадратичная ошибка  My_Gradient_Boosting_Regressor: {mse_reg:.4f}")

Среднеквадратичная ошибка  My_Gradient_Boosting_Regressor: 584.7498


In [25]:
X_train, X_test, y_train, y_test = upgraded_classification(df_class)

In [28]:
def search_params(type_task, my_model,
                  X_train, X_test, y_train, y_test,
                  n_estimators=(250, 500),
                  max_depth=(3, 7)):

    best_score = -1e18 if type_task == 'clf' else 1e18
    best_params = None

    for n_est in n_estimators:
        for depth in max_depth:
              model = my_model(
                  n_estimators=n_est,
                  max_depth=depth
              )

              model.fit(X_train, y_train)
              preds = model.predict(X_test)

              if type_task == 'clf':
                  score = accuracy_score(y_test, preds)
                  if score > best_score:
                      best_score = score
                      best_params = (n_est, depth)

              else:
                  score = mean_squared_error(y_test, preds)
                  if score < best_score:
                      best_score = score
                      best_params = (n_est, depth)

    return best_score, best_params


In [29]:
acc, best_params = search_params('clf',My_Gradient_Boosting_Classifier,X_train,X_test,y_train,y_test)
print(f"Лучшая точность My_Gradient_Boosting_Classifier: {acc:.4f}")
print(f"Best params: n_estimators={best_params[0]}, max_depth={best_params[1]}")

Лучшая точность My_Gradient_Boosting_Classifier: 0.8549
Best params: n_estimators=250, max_depth=7


In [30]:
X_train, X_test, y_train, y_test = upgraded_regression(df_reg)

In [31]:
mse, best_params = search_params('reg',My_Gradient_Boosting_Regressor,X_train,X_test,y_train,y_test)
print(f"Лучшая среднеквадратичная ошибка My_Gradient_Boosting_Regressor: {mse:.4f}")
print(f"Best params: n_estimators={best_params[0]}, max_depth={best_params[1]}")

Лучшая среднеквадратичная ошибка My_Gradient_Boosting_Regressor: 461.6550
Best params: n_estimators=500, max_depth=7


Стандартный бейзлайн:

Библиотечная реализация:
* Точность GradientBoostingClassifier: 0.8899
* Среднеквадратичная ошибка GradientBoostingRegressor: 582.0431

Имплементация алгоритма:
* Точность My_Gradient_Boosting_Classifier: 0.8578
* Среднеквадратичная ошибка My_Gradient_Boosting_Regressor: 584.7498

Улучшенный байзлайн:

Библиотечная реализация:
* Точность GradientBoostingClassifier: 0.8715
* Среднеквадратичная ошибка GradientBoostingRegressor: 504.212

Имплементация алгоритма:
* точность My_Gradient_Boosting_Classifier: 0.8549
* среднеквадратичная ошибка My_Gradient_Boosting_Regressor: 461.6550

Итог: Улучшенный бейзлайн положительно сказался на качестве регрессионных моделей в обоих подходах, особенно в собственной реализации.

Для классификации наблюдается незначительное снижение точности у библиотечных моделей, тогда как собственная реализация остаётся стабильной.

Библиотечные реализации Gradient Boosting показывают высокую точность и надёжность, однако собственная имплементация, при правильной настройке, также способна достичь значительного улучшения MSE для регрессии.